## Let's use sigpy

In [ ]:
# ! conda install -c frankong sigpy # didn't work so I just pip-installed sigpy

In [2]:
# ! salloc.tinygpu --gres=gpu:1 --time=12:00:00
from sigpy.mri.app import EspiritCalib as Espirit
import numpy as np
import sigpy as sp
import h5py as h5

/home/hernan/Documents/Master/WiSe22/CIP/CIP_-MoDL_vs_VarNet-/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Args:
 
        * ksp (array): k-space array of shape [num_coils, n_ndim, ..., n_1]

        * calib (tuple of ints): length-2 image shape.

        * thresh (float): threshold for the calibration matrix.

        * kernel_width (int): kernel width for the calibration matrix.

        * max_power_iter (int): maximum number of power iterations.

        * device (Device): computing device.
        
        * crop (int): cropping threshold.


In [5]:
orgFilename = 'file_brain_AXFLAIR_201_6002910.h5'

folder = "/home/hernan/Documents/Master/WiSe22/CIP/CIP_-MoDL_vs_VarNet-/multicoil_train/"
with h5.File(folder+ orgFilename, 'r') as f:
    ksp = f['kspace'][()] # slices, coils, h, w
    #ksp = np.transpose(ksp, (0,2,3,1)) 
    #ksp = np.swapaxes(ksp, 0,1) # -> coils, slices, h, w
    #csm = Espirit(ksp, device=sp.Device(0)).run() # CuPy needed
    scm = []
    for s in range(ksp.shape[0]):
        #k1 = ksp[:, s, ...]
        #c1 = Espirit(k1, device=sp.Device(0)).run() # CuPy needed
        c1 = Espirit(ksp).run() # CuPy needed
        scm.append(c1)

    scm = np.array(scm)

: 

: 

In [5]:
def saveCSM(csm: np.ndarray, name: str, mode = 'train'):
    with h5.File(name, 'w') as target: # creating a new file containing the necesary data to use it with MoDL
            if mode == 'train':
                #target.create_dataset('atb', data = atb_data)    # is the aliased/noisy image # It is not necessary for Brain_data
                target.create_dataset('trnCsm', data = csm)    # saves the coil sensitivity maps
            elif mode == 'test':
                #target.create_dataset('atb', data = atb_data)    # is the aliased/noisy image # It is not necessary for Brain_data
                target.create_dataset('tstCsm', data = csm)    # saves the coil sensitivity maps

In [7]:
filename = 'csm_' + orgFilename
saveCSM(scm, filename)

In [ ]:
import matplotlib.pyplot as plt
plt.show()

In [ ]:
def save(ground_truth: np.ndarray, csm: np.ndarray, mask: np.ndarray, name: str, mode = 'train'):
    with h5.File(name, 'w') as target: # creating a new file containing the necesary data to use it with MoDL
            # order: <KeysViewHDF5 ['atb', 'csm', 'mask', 'org']>
            if mode == 'train':
                #target.create_dataset('atb', data = atb_data)    # is the aliased/noisy image # It is not necessary for Brain_data
                target.create_dataset('trnCsm', data = csm)    # saves the coil sensitivity maps
                target.create_dataset('trnMask', data = mask)   # is the undersampling mask 
                target.create_dataset('trnOrg', data = ground_truth)#f['reconstruction_rss'])    # this is the original ground truth
            elif mode == 'test':
                #target.create_dataset('atb', data = atb_data)    # is the aliased/noisy image # It is not necessary for Brain_data
                target.create_dataset('tstCsm', data = csm)    # saves the coil sensitivity maps
                target.create_dataset('tstMask', data = mask)   # is the undersampling mask 
                target.create_dataset('tstOrg', data = ground_truth)#f['reconstruction_rss'])    # this is the original ground truth